- 0番目から始まる8文字読み込んで、nを得る
- 8番目から始まるn文字を読んで、XYZRPYを得る
- 8+n番目から始まる残り全部を読んで、imgを得る

In [2]:
import numpy as np
import glob
import cv2

XYZ = np.load("/root/share/temp/aruco/data/XYZ.npy")
RPY = np.load("/root/share/temp/aruco/data/RPY.npy")

frames = []
# for file in sorted(glob.glob("/root/share/temp/aruco/data/orig/shisa*png")):
for file in sorted(glob.glob("/root/share/temp/aruco/data/crop/shisa*png")):

    frame = cv2.imread(file, cv2.IMREAD_UNCHANGED)
    # frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2RGBA)
    frames.append(frame)

print(len(XYZ), len(RPY), len(frames))

94 94 94


In [ ]:
import socket

t = 0
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind(('0.0.0.0', 5900))
    s.listen(1)
    while True:
        conn, addr = s.accept()
        print("ac", end=" ")
        with conn:
            while True:
                data = conn.recv(1024)
                if not data:
                    break
                # print(data, end=" ")  # byte
                img_str = cv2.imencode(".png", frames[t])[1].tostring()

                msg = ("{} " * 6)[:-1].format(*XYZ[t], *RPY[t]).encode()  # [:-1]
                msglen = len(msg)
                msg = "{:04}    ".format(msglen).encode() + msg
                conn.sendall(msg + img_str)

                t += 1
                t %= len(XYZ)

ac ac 

In [3]:
len(img_str)

3186

In [10]:
import moviepy.editor as mpy
frames2 = [cv2.cvtColor(frame, cv2.COLOR_RGB2BGR) for frame in frames]
clip = mpy.ImageSequenceClip(frames2, fps=15)
clip.write_videofile("send_orig.mp4", fps=15)

Moviepy - Building video send_orig.mp4.
Moviepy - Writing video send_orig.mp4



Moviepy - Done !
Moviepy - video ready send_orig.mp4
